In [ ]:
!pip install transformers datasets

In [ ]:
from datasets import load_dataset
dataset=load_dataset("imdb")


In [ ]:
print(dataset)

In [ ]:
dataset['train'][1]

In [ ]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained('bert-base-cased')

def tokenize_function(examples):
  return tokenizer(examples['text'],padding='max_length',truncation=True, max_length=200)
  #return tokenizer(examples['text'],padding=True,truncation=True)


In [ ]:
tokenized_datasets=dataset.map(tokenize_function,batched=True)  

In [ ]:
print(tokenized_datasets)

In [ ]:
small_train=tokenized_datasets['train'].shuffle(seed=42).select(range(1000))
small_eval=tokenized_datasets['test'].shuffle(seed=42).select(range(1000))

In [ ]:
print(small_train)

In [ ]:
small_train[5]['text']

In [ ]:
print(len(small_train[5]['input_ids']))

In [ ]:
print(len(small_train[5]['input_ids']))

In [ ]:
print(small_train[5]['input_ids'])

In [ ]:
print(small_train[5]['attention_mask'])

In [ ]:
tokenizer.decode(small_train[5]['input_ids'])

In [ ]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

model=TFAutoModelForSequenceClassification.from_pretrained('bert-base-cased',num_labels=2)
tf_train=small_train.remove_columns(['text']).with_format('tensorflow')
tf_eval=small_eval.remove_columns(['text']).with_format('tensorflow')

train_features={x:tf_train[x] for x in tokenizer.model_input_names}
train_tf_dataset=tf.data.Dataset.from_tensor_slices((train_features,tf_train['label']))
train_tf_dataset=train_tf_dataset.shuffle(len(tf_train)).batch(8)

eval_features={x:tf_eval[x] for x in tokenizer.model_input_names}
eval_tf_dataset=tf.data.Dataset.from_tensor_slices((train_features,tf_eval['label']))
eval_tf_dataset=eval_tf_dataset.shuffle(len(tf_eval)).batch(8)

In [ ]:
train_tf_dataset

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=tf.metrics.SparseCategoricalAccuracy(),
)

In [ ]:
model.fit(train_tf_dataset,validation_data=eval_tf_dataset,epochs=1)